In [1]:
!pip install --upgrade torch==1.7.1 torchtext==0.8.1 torchvision==0.8.2

     |████████████████████████████████| 776.8MB 20kB/s 
     |████████████████████████████████| 7.0MB 36.4MB/s 
     |████████████████████████████████| 12.8MB 6.2MB/s 
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 1.6MB 5.1MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (56.0.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 4.1 MB/s 
     |████████████████████████████████| 9.1 MB 55.0 MB/s 
     |████████████████████████████████| 1.1 MB 51.4 MB/s 
     |████████████████████████████████| 42 kB 932 kB/s 
     |████████████████████████████████| 456 kB 66.0 MB/s 
     |████████████████████████████████| 113 kB 73.3 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=604e7ff8fb5adec0baf9b5e7e1c91b78cf278dc643caec8969f13225611a0a59
  Stored in directory: /root/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attem

In [3]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
#import pdbf

In [25]:
from google.colab import files
uploaded = files.upload()

Saving text_dataset.csv to text_dataset.csv


In [5]:
!unzip archive.zip

Archive:  archive.zip
  inflating: test.txt                
  inflating: train.txt               
  inflating: val.txt                 


In [6]:
#dataset_full = pd.read_csv('text_dataset.csv')

In [7]:
#dataset_full

In [26]:
data_train_csv = pd.read_csv('train.txt', sep=';', names = ['caption', 'sentiment'])
data_train_csv.to_csv('train.csv')

In [27]:
data_train_csv

,caption,sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [28]:
data_test_csv = pd.read_csv('test.txt', sep=';', names = ['caption', 'sentiment'])
data_test_csv.to_csv('test.csv')

In [29]:
data_test_csv

,caption,sentiment
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
1995,i just keep feeling like someone is being unki...,anger
1996,im feeling a little cranky negative after this...,anger
1997,i feel that i am useful to my people and that ...,joy
1998,im feeling more comfortable with derby i feel ...,joy


In [30]:
data_validation_csv = pd.read_csv('val.txt', sep=';', names = ['caption', 'sentiment'])
data_validation_csv.to_csv('val.csv')

In [31]:
data_validation_csv

,caption,sentiment
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


Now joining all the 3 given databases into one

In [32]:
complete_dataset_csv = data_train_csv
complete_dataset_csv = complete_dataset_csv.append(data_test_csv)
complete_dataset_csv = complete_dataset_csv.append(data_validation_csv)
complete_dataset_csv.to_csv('complete_dataset.csv')
complete_dataset_csv.size

40000

Emotions before editing:

In [33]:
complete_dataset_csv.sentiment.unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

Mapping:


0.   sad
1.   anger
2.   fear
3.   happy



In [34]:
complete_dataset_csv['sentiment'] = complete_dataset_csv['sentiment'].replace(['sadness','joy','anger','fear'],['0','3','1','2'])

1-> angry, 3-> happy, 0-> sad, 2->fear

In [35]:
complete_dataset_csv[:10]

,caption,sentiment
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,1
5,ive been feeling a little burdened lately wasn...,0
6,ive been taking or milligrams or times recomme...,surprise
7,i feel as confused about life as a teenager or...,2
8,i have been with petronas for years i feel tha...,3
9,i feel romantic too,love


In [36]:
complete_dataset_csv = complete_dataset_csv[complete_dataset_csv.sentiment != 'surprise']
complete_dataset_csv = complete_dataset_csv[complete_dataset_csv.sentiment != 'love']


In [37]:
complete_dataset_csv.sentiment.unique()

array(['0', '1', '2', '3'], dtype=object)

In [38]:
complete_dataset_csv.size

35280

In [39]:
complete_dataset_csv.to_csv('complete_dataset_final_2.csv')

sentiments: 0 for angry; 1 for happy; 2 for sad; 3 for sleepy

In [43]:
class dataset_cat_sentiments(torch.utils.data.Dataset):

  def __init__(self, csv_data, vocab=None):
    self.csv_dataset = pd.read_csv(csv_data)
    self.tokenizer = torchtext.data.utils.get_tokenizer('spacy', 'en_core_web_sm')
    toks = self.tokenizer(" ".join(self.csv_dataset.caption.unique()))

    if vocab is None:
            self.vocab = torchtext.vocab.build_vocab_from_iterator([toks])
    else:
            self.vocab = vocab
    self.toks = torch.LongTensor([self.vocab[tok] for tok in toks])
    self.LABELS = self.csv_dataset.sentiment.unique()
    self.LENGTH = self.csv_dataset.shape[0]
    self.label_dict = dict(zip(self.LABELS,range(len(self.LABELS))))

    self.dataset_dict = []
    for i in range(self.LENGTH):
      labels_ret = self.label_dict[self.csv_dataset.iloc[i].sentiment]
      toks = self.tokenizer(self.csv_dataset.iloc[i].caption)
      toks_ret = torch.LongTensor([self.vocab[tok] for tok in toks])
      self.dataset_dict.append((labels_ret, toks_ret))
    
    
  def get_labels(self):
    return set([(i) for i in range(len(self.LABELS))])

  def get_vocab(self):
    return self.vocab

  def DATASETS(self):
    return self.dataset_dict[:int(self.LENGTH*8/10)], self.dataset_dict[int((self.LENGTH*(8/10))+1):]

  def __len__(self):
    return self.LENGTH

  def __getitem__(self, i):
    
    # labels_ret = self.label_dict[self.csv_dataset.iloc[i].sentiment]
    # toks = self.tokenizer(self.csv_dataset.iloc[i].text)
    # toks_ret = torch.LongTensor([self.vocab[tok] for tok in toks])
    # return (labels_ret, toks_ret)
    return self.dataset_dict[i]

In [45]:
ap = dataset_cat_sentiments('complete_dataset_final_2.csv')
ap.LABELS
ap.get_labels()
ap

1lines [00:00, 19.83lines/s]


In [46]:
train_ds, test_ds = ap.DATASETS()

In [47]:
vb = ap.get_vocab()
len(vb)
vb.itos[:10]

['<unk>', '<pad>', 'i', 'feel', 'and', 'to', 'the', 'a', 'feeling', 'that']

In [48]:
vb.stoi['should']

114

In [49]:
xs, ys = ap[0]
' '.join([ap.vocab.itos[x] for x in ys])

'i did nt feel humiliated'

In [50]:
def mk_offsets(samples):
    txts = [txt for _, txt in samples]
    lens = [0] + [len(txt) for txt in txts]
    offs = torch.tensor(lens[:-1]).cumsum(0)
    labs = torch.tensor([lab for lab, txt in samples])
    txts = torch.cat(txts)
    return labs, txts, offs

In [51]:
mk_offsets(test_ds[0:4])

(tensor([3, 0, 3, 0]),
 tensor([   2,    3, 1685,   21,    2,  166,    9,    2,   23,   91,   40, 1711,
            4,  577,    2,    3,  508,   38,    2,   14,   78,  789,    2,    3,
           44,  309,    9,    2,   30,   29,  160,   15,    6, 8813,   50,   80,
            9,   45,   16,  613,   92,  303, 2097,    2,   19,  266,    5,  805,
         3238,   11,  126,   10,    7,  463,  415,   10,   11,  389,    4,   19,
          509,   13,   38,    2,   19,  193,  126,   12,    6,  463, 6441]),
 tensor([ 0, 14, 22, 43]))

In [52]:
class TextCats(nn.Module):
    def __init__(self, n_words, emb_dim, n_cats):
        super().__init__()
        self.embedding = nn.EmbeddingBag(n_words, emb_dim, mode='mean', sparse=True)
        self.fc = nn.Linear(emb_dim, n_cats)
        nn.init.xavier_uniform_(self.embedding.weight.data)
        nn.init.xavier_uniform_(self.fc.weight.data)
    def forward(self, text, offs):
        emb = self.embedding(text, offs)
        return self.fc(emb)

In [53]:
device = torch.device('cpu')

def run_test(model, ds, crit):
    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=16, collate_fn=mk_offsets)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='test iter'):
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        with torch.no_grad():
            outs = model(txts, offs)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
    return total_loss / len(ds), total_acc / len(ds)

def run_train(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=16, collate_fn=mk_offsets, shuffle=True)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='train iter'):
        opt.zero_grad()
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        outs = model(txts, offs)
        loss = crit(outs, labs)
        loss.backward()
        opt.step()
        total_loss += loss.item()
        total_acc += (outs.argmax(1) == labs).sum().item()
    sched.step()
    return total_loss / len(ds), total_acc / len(ds)

def run_all(model, test_ds, train_ds, crit, opt, sched, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs'):
        train_loss, train_acc = run_train(model, train_ds, crit, opt, sched)
        test_loss, test_acc = run_test(model, test_ds, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f} acc {train_acc:.4f}   test loss {test_loss:.6f} acc {test_acc:.4f}') 

In [54]:
model = TextCats(len(vb), 50, 4)
model.to(device);

In [55]:
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=3.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.7)

In [56]:
run_all(model, test_ds, train_ds, crit, opt, sched, 10) 

epoch 0   train loss 0.071294 acc 0.4976   test loss 0.044957 acc 0.7522


epoch 1   train loss 0.028210 acc 0.8440   test loss 0.021130 acc 0.8917


epoch 2   train loss 0.014801 acc 0.9295   test loss 0.018799 acc 0.9050


epoch 3   train loss 0.009913 acc 0.9577   test loss 0.014812 acc 0.9269


epoch 4   train loss 0.007621 acc 0.9724   test loss 0.014957 acc 0.9217


epoch 5   train loss 0.006348 acc 0.9787   test loss 0.013623 acc 0.9286


epoch 6   train loss 0.005527 acc 0.9833   test loss 0.013916 acc 0.9240


epoch 7   train loss 0.005089 acc 0.9863   test loss 0.013523 acc 0.9305


epoch 8   train loss 0.004807 acc 0.9878   test loss 0.013431 acc 0.9317


epoch 9   train loss 0.004591 acc 0.9887   test loss 0.013308 acc 0.9300




Testing out the model:
---



1-> angry, 3-> happy, 0-> sad, 2->fear

In [57]:
lst = ["i am felling grouchy"]
str_arr = (lst[0].split())
str_index = []
str_index2 = []
str_index3 = []
for i in range(len(str_arr)):
  str_index.append(vb.stoi[str_arr[i]])
  
  if(i+1<len(str_arr)):
    str_index2.append(vb.stoi[str_arr[i]+' '+str_arr[i+1]])
  
tensor_in = torch.LongTensor(str_index)
model.eval()
outs = model(tensor_in, torch.LongTensor([0]))
outs.argmax(1)
# vb.stoi['hello', 'cat', 'is', 'very', 'sad']

tensor([1])

Optimizing our model for mobile and downloading it:


In [58]:
from torch.utils.mobile_optimizer import optimize_for_mobile

In [59]:
MemeSentiment_model = torch.jit.script(model)

In [60]:
torchscript_model_optimized = optimize_for_mobile(MemeSentiment_model)
torch.jit.save(torchscript_model_optimized, "MemeSentiment_model.pt")

Downloading the vocab

In [61]:
def save_vocab(vocab, path):
    with open(path, 'w+') as f:
        for token, index in vocab.stoi.items():
            f.write(f'{token}' +',' + f'{index} \n')

In [62]:
save_vocab(vb,'vocab.txt')